In [6]:
### Different approach - batch size learning 
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import shuffle
import random

import sys
sys.path.append("..")
import fede.utils
from fede.fedavg import Fedavg
from fede.fed_transfer import Fed_Avg_Client
from fede.supported_modles import Supported_modles

In [7]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [8]:
fedavg = Fedavg("global")
lr = 0.05
fedavg.init_global_model(lr, Supported_modles.SGD_classifier, None,78)
# applicable_clients = fedavg.wait_for_data(3)

In [9]:
# applicable_clients

In [10]:
selected_model = Supported_modles.SGD_classifier
number_of_rounds = 3
batch_size = 0.05
epochs = 10
# prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    # applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0

    applicable_clients = fedavg.wait_for_data(3)
    
    for client in applicable_clients:

        print(f'Client name: {client.name}')
        
        fedavg.load_global_model(client.model, selected_model) #load global model on the client model

        fedavg.train_local_agent(client.X_train, client.y_train, client.model, epochs, client.sample_weights, selected_model) #make partial fit on globsl model

        round_weights.append(client.X_train.shape[0])
        dataset_size += client.X_train.shape[0]
        print(round_weights)
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)
    print(fedavg.model.intercept_)

Starting new round!
Server is Listening.....
Data received from client1
Data received from client2
Data received from client3
Client name: client1
[8954]
Client name: client1
[8954, 8226]
Client name: client1
[8954, 8226, 8244]
[-2982.34814272]
Starting new round!
Server is Listening.....
Data received from client1
Data received from client2
Data received from client3
Client name: client1
[8954]
Client name: client1
[8954, 8226]
Client name: client1
[8954, 8226, 8244]
[-6555.19546864]
Starting new round!
Server is Listening.....
Data received from client1
Data received from client2
Data received from client3
Client name: client1
[8954]
Client name: client1
[8954, 8226]
Client name: client1
[8954, 8226, 8244]
[-8651.42657597]


In [ ]:
fedavg.sendmodel()